In [1]:
import pandas as pd
import numpy as np
import os
directory = 'modelos/st'
if not os.path.exists(directory):
    os.makedirs(directory)
# Guardo modelo como un pickle file 
from joblib import dump 

In [2]:
auctions = pd.read_csv('../datos/auctions.csv', low_memory=False)

# Preparo el df y divido ventanas

In [3]:
auctions['date'] = pd.to_datetime(auctions['date'])

In [4]:
w1 = auctions[auctions['date'].dt.day < 21]
w2 = auctions[(auctions['date'].dt.day > 18) & (auctions['date'].dt.day < 22)]
w3 = auctions[(auctions['date'].dt.day > 19) & (auctions['date'].dt.day < 23)]
w4 = auctions[(auctions['date'].dt.day > 20) & (auctions['date'].dt.day < 24)]
w5 = auctions[(auctions['date'].dt.day > 21) & (auctions['date'].dt.day < 25)]
w6 = auctions[(auctions['date'].dt.day > 22) & (auctions['date'].dt.day < 26)]
w7 = auctions[(auctions['date'].dt.day > 23) & (auctions['date'].dt.day < 27)]

In [5]:
windowStart = pd.to_datetime("2019-04-18 00:00:00.000000")
w1['timeFromWindowStart'] = (w1['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-19 00:00:00.000000")
w2['timeFromWindowStart'] = (w2['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-20 00:00:00.000000")
w3['timeFromWindowStart'] = (w3['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-21 00:00:00.000000")
w4['timeFromWindowStart'] = (w4['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-22 00:00:00.000000")
w5['timeFromWindowStart'] = (w5['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-23 00:00:00.000000")
w6['timeFromWindowStart'] = (w6['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-23 00:00:00.000000")
w7['timeFromWindowStart'] = (w7['date'] - windowStart)/np.timedelta64(1,'s')

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [6]:
w1 = w1.sort_values(by=['device_id', 'date'])
w1 = w1.drop_duplicates('device_id', keep = 'first')
w2 = w2.sort_values(by=['device_id', 'date'])
w2 = w2.drop_duplicates('device_id', keep = 'first')
w3 = w3.sort_values(by=['device_id', 'date'])
w3 = w3.drop_duplicates('device_id', keep = 'first')
w4 = w4.sort_values(by=['device_id', 'date'])
w4 = w4.drop_duplicates('device_id', keep = 'first')
w5 = w5.sort_values(by=['device_id', 'date'])
w5 = w5.drop_duplicates('device_id', keep = 'first')
w6 = w6.sort_values(by=['device_id', 'date'])
w6 = w6.drop_duplicates('device_id', keep = 'first')
w7 = w7.sort_values(by=['device_id', 'date'])
w7 = w7.drop_duplicates('device_id', keep = 'first')

In [7]:
w1.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w2.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w3.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w4.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w5.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w6.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w7.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']

In [8]:
w1 = w1.drop(columns = ['date'])
w2 = w2.drop(columns = ['date'])
w3 = w3.drop(columns = ['date'])
w4 = w4.drop(columns = ['date'])
w5 = w5.drop(columns = ['date'])
w6 = w6.drop(columns = ['date'])
w7 = w7.drop(columns = ['date'])

## Aplico KNN y RandomForest 
Aplico KNN y RandomForest en la primer ventana y hago las predicciones de la cuarta ventana y despues le voy a agregar esas predicciones como features

In [9]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [10]:
X, y = w1.iloc[:,:-1],w1.iloc[:,-1]

In [11]:
neigh = KNeighborsRegressor(n_neighbors=101)

In [12]:
neigh.fit(X, y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=101, p=2,
                    weights='uniform')

In [13]:
preds_KNN_w4 = neigh.predict(w4.iloc[:,:-1])

In [14]:
rf = RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=5, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False)

In [15]:
rf.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [16]:
preds_RF_w4 = rf.predict(w4.iloc[:,:-1])

## Entreno con w4
Ya me guarde las predicciones de w4 de RF y de KNN y las voy a agregar como features luego.
Ahora voy a entrenar con w4 y validar con w7 y luego les voy a agregar tanto a w4 como a w7 los features de las predicciones de KNN y de RF

In [17]:
X, y = w4.iloc[:,:-1],w4.iloc[:,-1]

In [18]:
neigh = KNeighborsRegressor(n_neighbors=101)

In [19]:
neigh.fit(X, y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=101, p=2,
                    weights='uniform')

In [20]:
preds_KNN_w7 = neigh.predict(w7.iloc[:,:-1])

In [21]:
rf = RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=5, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False)

In [22]:
rf.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [23]:
preds_RF_w7 = rf.predict(w7.iloc[:,:-1])

In [24]:
w4["pred_KNN"] = preds_KNN_w4

In [25]:
w4["pred_RF"] = preds_RF_w4
w7["pred_KNN"] = preds_KNN_w7
w7["pred_RF"] = preds_RF_w7

In [26]:
w4.columns = ['device_id', 'ref_type_id', 'source_id', 'pred_KNN', 'pred_RF', 'st']
w7.columns = ['device_id', 'ref_type_id', 'source_id', 'pred_KNN', 'pred_RF', 'st']

In [27]:
w4.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
36601349,40621409780134,1,0,226857.222979,82963.592845,78657.289438
33917684,168103949904656,1,3,173044.526198,82963.592845,80921.079159
9402375,186034136943920,1,1,88.851558,82963.592845,79575.202337
13736755,295841792051458,1,1,432.827168,82963.592845,79575.202337
43932908,345999128501141,1,0,164703.549846,82963.592845,78657.289438


In [28]:
w7.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
39675750,69039685746313,1,7,212658.597103,89392.758783,87348.172454
1528620,345999128501141,1,2,336762.048531,89392.758783,88099.527648
20706804,360710529886978,1,1,133087.256609,89392.758783,79298.492452
35163923,365882020742330,1,8,183027.763854,89392.758783,190667.559405
20794330,416301579449694,1,1,98969.446617,89392.758783,79298.492452


## Ahora voy a entrenar w4 y validar con w7 con XBBoost con las nuevas features

In [29]:
X, y = w4.iloc[:,:-1],w4.iloc[:,-1]

In [30]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [31]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.9, learning_rate = 0.80,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [32]:
xg_reg.fit(X,y)

[03:38:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0,
             importance_type='gain', learning_rate=0.8, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [33]:
preds = xg_reg.predict(w7.iloc[:,:-1])

In [34]:
rmse = np.sqrt(mean_squared_error(w7.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 7268.563338


## Preparo el target_competencia

In [36]:
targets_st = pd.read_csv('../datos/target_competencia_ids.csv')
targets_st = targets_st[~targets_st['ref_hash'].str.contains('_sc')]
targets_st['ref_hash'] = targets_st['ref_hash'].map(lambda x: str(x)[:-3])

In [37]:
targets_st['ref_hash'] = targets_st['ref_hash'].astype(np.int64)
targets_st.columns = ['device_id', 'date_dif']

In [38]:
auctions_without_duplicates = auctions.drop_duplicates(subset = 'device_id')

In [39]:
targets_st = targets_st.merge(auctions_without_duplicates[['device_id', 'ref_type_id', 'source_id']], on = 'device_id', how = 'left')

In [40]:
targets_st = targets_st.drop(columns = ['date_dif'])

## Predigo con KNN y con RF target compentencia para agregárcelos como features

In [41]:
w7 = w7.drop(columns = ['pred_KNN'])
w7 = w7.drop(columns = ['pred_RF'])

In [42]:
w7.head()

,device_id,ref_type_id,source_id,st
39675750,69039685746313,1,7,87348.172454
1528620,345999128501141,1,2,88099.527648
20706804,360710529886978,1,1,79298.492452
35163923,365882020742330,1,8,190667.559405
20794330,416301579449694,1,1,79298.492452


In [43]:
X, y = w7.iloc[:,:-1],w7.iloc[:,-1]

In [44]:
neigh = KNeighborsRegressor(n_neighbors=101)

In [45]:
neigh.fit(X, y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=101, p=2,
                    weights='uniform')

In [46]:
preds_KNN_targets = neigh.predict(targets_st)
dump(neigh, 'modelos/st/neigh.pkl')

['modelos/st/neigh.pkl']

In [47]:
rf = RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=5, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False)

In [48]:
rf.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [49]:
preds_RF_targets = rf.predict(targets_st)
dump(rf, 'modelos/st/rf.pkl')

['modelos/st/rf.pkl']

In [50]:
w7["pred_KNN"] = preds_KNN_w7
w7["pred_RF"] = preds_RF_w7
targets_st["pred_KNN"] = preds_KNN_targets
targets_st["pred_RF"] = preds_RF_targets

In [51]:
w7.columns = ['device_id', 'ref_type_id', 'source_id', 'pred_KNN', 'pred_RF', 'st']

In [52]:
w7.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
39675750,69039685746313,1,7,87348.172454,89392.758783,87348.172454
1528620,345999128501141,1,2,88099.527648,89392.758783,88099.527648
20706804,360710529886978,1,1,79298.492452,89392.758783,79298.492452
35163923,365882020742330,1,8,190667.559405,89392.758783,190667.559405
20794330,416301579449694,1,1,79298.492452,89392.758783,79298.492452


In [53]:
targets_st.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF
0,1000169251625791246,1,1,81316.749726,79298.492452
1,1000395625957344683,1,8,81094.796346,92193.074740
2,1003027494996471685,1,0,81768.118050,83109.410677
3,1006670001679961544,7,0,80504.595306,83109.410677
4,1007573308966476713,1,1,81028.109010,79298.492452


## Ahora aplico XGBoost entrendando con w7 y prediciendo targets

In [54]:
X, y = w7.iloc[:,:-1],w7.iloc[:,-1]

In [55]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/gabriel/workspace/7506-1c2019/env/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [56]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.9, learning_rate = 0.80,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [57]:
xg_reg.fit(X,y)

[10:35:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0,
             importance_type='gain', learning_rate=0.8, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [58]:
preds = xg_reg.predict(targets_st)
dump(xg_reg, 'modelos/st/xg_reg.pkl')

['modelos/st/xg_reg.pkl']

In [59]:
for pred in preds: print(pred)

79299.82
79931.26
83127.5
80217.84
79299.82
83127.5
83134.82
82209.48
82209.48
79299.82
79299.82
79708.9
80334.48
83127.5
82209.48
79218.19
79709.84
80217.84
82216.8
83127.5
82619.5
80334.48
82209.48
82209.48
87877.23
79299.82
79298.875
80217.84
79299.82
79299.82
82209.48
82209.48
79298.875
84935.99
83127.5
82209.48
83127.5
82209.48
83134.82
82619.5
83127.5
82209.48
82209.48
79299.82
79709.84
81579.45
82208.53
79299.82
83127.5
79299.82
84935.05
80217.84
82619.5
80217.84
82744.44
82209.48
82209.48
82209.48
82209.48
82209.48
79299.82
79299.82
77424.81
79299.82
80217.84
80334.48
82208.53
79299.82
80217.84
79299.82
79299.82
79709.84
79931.26
79299.82
79299.82
82209.48
79318.55
82209.48
82861.91
84935.99
82619.5
83127.5
83127.5
82209.48
79299.82
79299.82
82209.48
82619.5
83127.5
80334.48
77424.81
77424.81
82209.48
79834.77
82848.234
82209.48
83127.5
82209.48
77424.81
79299.82
82209.48
79299.82
84489.12
83127.5
84935.99
83127.5
80217.84
79299.82
82209.48
83127.5
84935.99
82209.48
83127.5
792

83127.5
79904.94
79299.82
79299.82
77424.81
79904.94
82026.336
79299.82
79299.82
77424.81
82619.5
79709.84
78536.34
80217.84
79299.82
80217.84
79299.82
79799.3
79299.82
79299.82
83134.82
80341.8
80334.48
83127.5
83127.5
82619.5
82209.48
79299.82
82209.48
83127.5
79620.75
82209.48
83127.5
82619.5
79299.82
79299.82
79299.82
79299.82
79299.82
82619.5
82209.48
84935.99
82209.48
82209.48
82209.48
79299.82
79709.84
80217.84
79298.875
79299.82
79904.94
79708.9
79299.82
79298.875
80334.48
79299.82
80217.84
82744.44
80217.84
82619.5
82209.48
82209.48
82619.5
79299.82
82209.48
82744.44
80217.84
83127.5
79299.82
82619.5
81446.01
82208.53
82209.48
77424.81
79299.82
78536.34
81579.45
77424.81
83134.82
83174.81
80217.84
79299.82
79299.82
82814.59
82209.48
83127.5
80378.36
81579.45
82209.48
82209.48
79299.82
82814.59
84489.12
79299.82
79709.84
79299.82
82209.48
82209.48
80334.48
82619.5
79299.82
80217.84
80341.8
83174.81
82619.5
80334.48
80334.48
79299.82
82744.44
79299.82
82126.9
82209.48
84954.72
8

79299.82
82156.54
82209.48
82209.48
82209.48
83127.5
83127.5
83127.5
79299.82
80217.84
82209.48
82209.48
82209.48
83127.5
82216.8
82209.48
79299.82
82156.54
80284.45
82530.41
83127.5
85066.195
85066.195
77424.81
79299.82
78536.34
79299.82
79299.82
83127.5
79299.82
79709.84
79299.82
79709.84
82209.48
79299.82
79299.82
82530.41
82209.48
82216.8
82126.9
82209.48
79299.82
84527.41
82209.48
82626.82
82216.8
79751.72
82209.48
79709.84
82209.48
83127.5
82209.48
82209.48
83127.5
82156.54
79299.82
83127.5
84527.41
79299.82
80217.84
79299.82
80217.84
80334.48
82209.48
79904.94
77424.81
77424.81
83127.5
82209.48
82209.48
80334.48
82209.48
82216.8
82209.48
79299.82
79709.84
77374.79
83127.5
79980.39
77374.79
77424.81
83127.5
82216.8
82216.8
79298.875
80217.84
81617.74
79299.82
82209.48
82209.48
83127.5
82208.53
82209.48
82209.48
82619.5
81617.74
79299.82
82209.48
82209.48
82209.48
82619.5
82208.53
79980.39
79299.82
82209.48
82619.5
82209.48
83127.5
82209.48
82209.48
83127.5
82209.48
82619.5
84527.